In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.7.1

openjdk version "1.8.0_352"
OpenJDK Runtime Environment (build 1.8.0_352-8u352-ga-1~20.04-b08)
OpenJDK 64-Bit Server VM (build 25.352-b08, mixed mode)


In [2]:
import sparknlp

spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

spark

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/opt/conda/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp-gpu_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-11ff8408-0705-43cd-aaf2-2a916979fd06;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.11;2.7.1 in central
	found com.typesafe#config;1.3.0 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-core;1.11.603 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.9 in central
	found org.apache.httpcomponents#httpcore;4.4.11 in central
	found commons-codec#commons-codec;1.11 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.faster

Spark NLP version 2.7.1
Apache Spark version: 2.4.4


In [3]:
# df = pd.read_csv('../input/twitter-class-balanced/balanced_data.csv')

In [4]:
# df = df.drop(columns='Unnamed: 0',axis=1)

In [5]:
# from sklearn.model_selection import train_test_split
# X = df['text']
# y = df['class']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [6]:
# train = list(zip(X_train, y_train))
# test = list(zip(X_test,y_test))
# df_train = pd.DataFrame(data = train,columns = ['description','category'])
# df_test = pd.DataFrame(data = test)
# df_train.to_csv('train_twitter')
# df_test.to_csv('test_twitter')

In [7]:
# ! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
# ! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv


In [8]:
df = spark.read \
      .option("header", True) \
      .csv("../input/twitter-separated/train_twitter.csv")
df.show(truncate=50)

+--------------------------------------------------+--------+
|                                       description|category|
+--------------------------------------------------+--------+
|she say she like me cuz ima real man . . . . . ...|       5|
|babygirl as jiggling dropping it low doing the ...|       1|
|my only talent is being able to do both the uwu...|       2|
|sobie ade buat postage rini pat pat jom all rea...|       2|
|i should make a video on that can it be top num...|       1|
|you can really pull off both cutesy and sexy huh ?|       2|
|yeah i m still trying to understand what the he...|       6|
|politics ha no place in professional sports . p...|       3|
|i would have if i wasn t spoiled by it s ending...|       4|
|it s always great to see the bivouac . it does ...|       5|
|bruh when i don t have the kid i m up so early ...|       4|
|this is so real tho i ll rlly be dying doing ha...|       7|
|infact i just noticed the views k am like who a...|       7|
|just se

In [9]:
from pyspark.sql.functions import col
df.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|       4|17343|
|       1|17315|
|       5|17294|
|       7|17272|
|       3|17240|
|       6|17208|
|       2|17184|
+--------+-----+



In [10]:
(train_df, val_df) = df.randomSplit([0.7, 0.3], seed = 8)
print("Training Dataset Count: " + str(train_df.count()))
print("Validation Dataset Count: " + str(val_df.count()))

Training Dataset Count: 84671
Validation Dataset Count: 36185


In [11]:
# actual content is inside description column
document = DocumentAssembler() \
.setInputCol("description") \
.setOutputCol("document") \
.setCleanupMode("shrink")

bert = BertSentenceEmbeddings.pretrained('sent_bert_base_cased') \
.setInputCols("document") \
.setOutputCol("bert_sentence_embeddings") \
.setLazyAnnotator(False)

# the classes/labels/categories are in category column
classifierdl = ClassifierDLApproach()\
.setInputCols(["bert_sentence_embeddings"])\
.setOutputCol("class")\
.setLabelColumn("category")\
.setMaxEpochs(4)\
.setLr(0.001)\
.setBatchSize(64)\
.setEnableOutputLogs(True)
#.setOutputLogsPath('logs')

pipeline = Pipeline(
    stages = [
        document,
        bert,
        classifierdl
    ])

sent_bert_base_cased download started this may take some time.


22/11/12 15:27:23 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/12 15:27:23 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information


Approximate size to download 389.1 MB
[ | ]

22/11/12 15:27:25 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/12 15:27:25 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/12 15:27:25 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http

sent_bert_base_cased download started this may take some time.


...

Approximate size to download 389.1 MB


22/11/12 15:27:26 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/12 15:27:26 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http client version >= 4.5.9 to avoid the breaking change introduced in apache client 4.5.7 and the latency in exception handling. See https://github.com/aws/aws-sdk-java/issues/1919 for more information
22/11/12 15:27:27 WARN ApacheUtils: NoSuchMethodError was thrown when disabling normalizeUri. This indicates you are using an old version (< 4.5.8) of Apache http client. It is recommended to use http

[ — ]Download done! Loading the resource.
[ / ]

2022-11-12 15:27:49.942845: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-11-12 15:27:49.989184: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2022-11-12 15:27:49.990949: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb73a58ac30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-11-12 15:27:49.990991: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-11-12 15:27:49.995484: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-11-12 15:27:50.084668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-12 15:27:50.

[OK!]


In [12]:
%%time
pipelineModel = pipeline.fit(train_df)

2022-11-12 15:28:04.265537: I tensorflow/cc/saved_model/reader.cc:31] Reading SavedModel from: /tmp/289531f5715f_classifier_dl4209532244932976113
2022-11-12 15:28:04.340762: I tensorflow/cc/saved_model/reader.cc:54] Reading meta graph with tags { serve }
2022-11-12 15:28:04.471925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-12 15:28:04.472987: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2022-11-12 15:28:04.473208: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_6

Training started - epochs: 4 - learning_rate: 0.001 - batch_size: 64 - training_examples: 84671 - classes: 7
Epoch 1/4 - 19.74s - loss: 2463.582 - acc: 0.2742579 - batches: 1323
Epoch 2/4 - 17.98s - loss: 2431.7117 - acc: 0.2997752 - batches: 1323
Epoch 3/4 - 18.62s - loss: 2424.0945 - acc: 0.3070322 - batches: 1323
Epoch 4/4 - 17.95s - loss: 2419.5562 - acc: 0.31063688 - batches: 1323
CPU times: user 1.39 s, sys: 223 ms, total: 1.61 s
Wall time: 2h 2min 47s


In [13]:
# get the predictions on validation Set

preds = pipelineModel.transform(val_df)

In [14]:
preds.select('description','category',"class.result").show(10, truncate=100)

[Stage 14:>                                                         (0 + 1) / 1]2022-11-12 17:30:49.964595: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-12 17:30:49.965651: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2022-11-12 17:30:49.965871: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib

+---------------------------------------------------------------------------------------------------+--------+------+
|                                                                                        description|category|result|
+---------------------------------------------------------------------------------------------------+--------+------+
|                              ! ! ! ! cool cool ! ! ! i will keep u posted when i start doing stuff|       5|   [2]|
|                                 ! ! ! ! the shiloh person omg ! ! i am bad at linear thought sorry|       4|   [4]|
|                                        ! ! ! ! they are the only reason i even want another season|       4|   [3]|
|! ! happy bday joanne i hope that you had a wonderful day and you were able to enjoy yourself ! ! !|       2|   [2]|
|                                        ! ! oops . it ha to be panpsychism made it out of my silo .|       6|   [7]|
|                                          ! ! yes ive b

In [15]:
preds_df = preds.select('description','category',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [19]:
preds_df

,description,category,result
0,! ! ! ! cool cool ! ! ! i will keep u posted w...,5,2
1,! ! ! ! the shiloh person omg ! ! i am bad at ...,4,4
2,! ! ! ! they are the only reason i even want a...,4,3
3,! ! happy bday joanne i hope that you had a wo...,2,2
4,! ! oops . it ha to be panpsychism made it out...,6,7
...,...,...,...
36180,zero wish lmao . i spent em all but i can get ...,6,3
36181,zimbabwe is a good example of a bad example . ...,7,5
36182,zoa is so pretty idk but she reminds of tzuyu ...,4,2
36183,zombie llama over here have a great rest of yo...,2,2


In [16]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))

              precision    recall  f1-score   support

           1       0.13      0.23      0.17      2873
           2       0.55      0.45      0.50      6364
           3       0.48      0.28      0.35      8920
           4       0.33      0.28      0.30      5943
           5       0.36      0.31      0.33      5968
           6       0.00      0.57      0.00         7
           7       0.29      0.25      0.27      6110

    accuracy                           0.31     36185
   macro avg       0.31      0.34      0.27     36185
weighted avg       0.39      0.31      0.34     36185



In [22]:
pipelineModel.save("./spark-NLP-bert")

22/11/12 18:31:56 WARN TaskSetManager: Stage 25 contains a task of very large size (209 KB). The maximum recommended task size is 100 KB.
